## Python で始める機器分析データの解析とケモメトリックス  
### 8.8. 部分最小二乗法 (PLS)

In [ ]:
import numpy
import pandas
from matplotlib import pyplot

In [ ]:
data = pandas.read_csv("data3.csv", header=0, index_col=0).T
data.index = pandas.read_csv("prop3.csv", header=0, index_col=0).iloc[:, 2].values
data = ((data.T - data.T.mean()) / data.T.std()).T
from scipy.signal import savgol_filter
buff = data
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
buff = savgol_filter(buff, 7, 2, 0)
buff = savgol_filter(buff, 7, 2, 1)
data = pandas.DataFrame(buff, index=data.index, columns=data.columns)
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size=0.6, random_state=8)

sklearn.cross_decomposition.PLSRegression  
https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
p1 = numpy.arange(2, 31, 1)
parm = {"n_components": p1}
search = GridSearchCV(PLSRegression(), parm, cv=3).fit(data.values, data.index)
pyplot.scatter(p1, search.cv_results_["mean_test_score"])
pyplot.show()
print(search.best_estimator_)
print(search.best_score_)
model = search.best_estimator_.fit(train.values, train.index)
calibration = model.predict(train.values)
validation = model.predict(test.values)

In [ ]:
pyplot.figure(figsize=(4, 4))
pyplot.plot([data.index.min(), data.index.max()], [data.index.min(), data.index.max()], c="black")
pyplot.scatter(train.index, calibration, label="calibration")
pyplot.scatter(test.index, validation, label="validation")
pyplot.legend()
pyplot.show()
from sklearn.metrics import mean_squared_error, r2_score
print("RMSEC =", numpy.sqrt(mean_squared_error(train.index, calibration)))
print("R^2 =", r2_score(train.index, calibration))
print("")
print("RMSEV =", numpy.sqrt(mean_squared_error(test.index, validation)))
print("R^2 =", r2_score(test.index, validation))

In [ ]:
spec = test.iloc[0]
print(model.predict([spec.values])[0][0])
print(((spec.values - model._x_mean) / model._x_std @ model.coef_ + model._y_mean)[0])

In [ ]:
pyplot.plot(data.columns, model.coef_)
pyplot.show()

(c) Shigeaki Morita